## Analysis of Time Tracking Data

The below graphs visualise my data from a time tracking app over a 3 week period. The aim is to gain a better understanding of where my time goes in terms of tasks and projects.

Some important points to note when considering these graphs:

- The Monday of the first week was a Bank Holiday in the UK
- These plots only represent the **time I entered into the app**, not the total time I worked.
  This is particularly noticeable in the first week when I wasn't very strict about logging everything.
  In the second and third weeks, I become more consistent.
- The specific tasks and projects I logged have been anonymised to protect myself (pleading the 5th!) and anyone I might have collaborated with from being identified.

In [ ]:
# Import required packages
import os
import fnmatch
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
from ipywidgets import interact

%matplotlib inline
cmap = matplotlib.cm.get_cmap("tab20")

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

In [ ]:
# Setting filepaths
#
# Get Current Working Directory
CWD = os.getcwd()

# Set Relative and Absolute filepaths for clean data
CLEAN_DATA_RELPATH = os.path.join(os.pardir, os.pardir, "data", "clean")
CLEAN_DATA_ABSPATH = os.path.join(CWD, CLEAN_DATA_RELPATH)

In [ ]:
# Find data files
datafiles = fnmatch.filter(
    os.listdir(CLEAN_DATA_ABSPATH),
    "[0-9]{4}-[0-9]{2}-[0-9]{2}_[0-9]{4}-[0-9]{2}-[0-9]{2}.csv",
)

In [ ]:
# Read in the first datafile
filepath = os.path.join(CLEAN_DATA_ABSPATH, datafiles[0])
data = pd.read_csv(
    filepath,
    converters={
        "dt_start": pd.to_datetime,
        "dt_end": pd.to_datetime,
        "Duration": pd.to_timedelta,
    },
    na_values=[0.0, "None"],
)

# Sort the data by start date
data.sort_values("dt_start", inplace=True, ignore_index=True)

# Convert `Duration` into hours
data["Duration"] = data["Duration"] / np.timedelta64(1, "h")

### Total time spent on tasks per week

In [ ]:
# Calculate total duration per task per week number
total_time_per_task = pd.DataFrame({})
for task in np.sort(data.Task.dropna().unique()):
    total_time_per_task[task] = (
        data[data.Task == task].groupby("week_num")["Duration"].sum()
    )

total_time_per_task["Total"] = data.groupby("week_num")["Duration"].sum()
total_time_per_task = total_time_per_task.T

In [ ]:
@interact
def plot_total_time_per_task(column=total_time_per_task.columns.tolist()):
    total_time_per_task[column].drop("Total", axis=0).sort_values().dropna().plot.pie(
        figsize=(12, 8),
        title=f"Total = {total_time_per_task.loc['Total', column]:.1f} hrs",
        ylabel="",
        colormap=cmap,
        autopct="%1.1f%%",
    )

    plt.legend(
        loc="upper left",
        bbox_to_anchor=(1.05, 1),
    );

### Total time spent on projects per week

In [ ]:
total_time_per_project = pd.DataFrame({})

for project in np.sort(data.Project.dropna().unique()):
    total_time_per_project[project] = (
        data[data.Project == project].groupby("week_num")["Duration"].sum()
    )

total_time_per_project["Total"] = data.groupby("week_num")["Duration"].sum()
total_time_per_project = total_time_per_project.T.sort_index()

In [ ]:
@interact
def plot_total_time_per_project(column=total_time_per_project.columns.tolist()):
    total_time_per_project[column].drop(
        "Total", axis=0
    ).sort_values().dropna().plot.pie(
        figsize=(12, 8),
        title=f"Total = {total_time_per_project.loc['Total', column]:.1f} hrs",
        ylabel="",
        colormap=cmap,
        autopct="%1.1f%%",
    )

    plt.legend(
        loc="upper left",
        bbox_to_anchor=(1.05, 1),
    );

### Total time spent on tasks per project (across all 3 weeks)

Due to the anonymisation process, this isn't the most useful mapping of tasks to project. However, consider this me pleading the 5th and not giving evidence against myself! Whether the evidence is damning or not is left as an exercise for the reader 😉

In [ ]:
columns = np.sort(data.Task.dropna().unique())
tasks_per_project = pd.DataFrame(columns=columns)

for project in np.sort(data.Project.dropna().unique()):
    tasks_per_project.loc[project] = (
        data[data.Project == project].groupby("Task")["Duration"].sum()
    )

tasks_per_project["Total"] = tasks_per_project.sum(axis=1)
tasks_per_project = tasks_per_project.T

In [ ]:
@interact
def plot_tasks_per_project(column=tasks_per_project.columns.tolist()):
    tasks_per_project[column].drop("Total", axis=0).sort_values().dropna().plot.pie(
        figsize=(12, 8),
        title=f"Total = {tasks_per_project.loc['Total', column]:.1f} hrs",
        ylabel="",
        colormap=cmap,
        autopct="%1.1f%%",
    )

    plt.legend(
        loc="upper left",
        bbox_to_anchor=(1.05, 1),
    );

### Tasks per Weekday

In [ ]:
# Create a MultiIndex for week number and day of week
MultiIndex = list(
    product(
        np.sort(data.week_num.dropna().unique()),
        days,
    )
)
index = pd.MultiIndex.from_tuples(MultiIndex, names=["week_num", "weekday"])

In [ ]:
columns = np.sort(data.Task.dropna().unique())
tasks_per_weekday = pd.DataFrame(columns=columns, index=index)

result = product(np.sort(data.week_num.dropna().unique()), days)

for i, (week_num, weekday) in enumerate(result):
    tasks_per_weekday.loc[week_num, weekday] = (
        data[(data.week_num == week_num) & (data.weekday == weekday)]
        .groupby("Task")["Duration"]
        .sum()
    )

tasks_per_weekday["Total"] = tasks_per_weekday.sum(axis=1)

In [ ]:
@interact
def plot_tasks_per_weekday(
    week_num=tasks_per_weekday.index.get_level_values("week_num").unique().tolist(),
):
    tasks_per_weekday.loc[week_num, :].drop("Total", axis=1).plot.bar(
        figsize=(12, 8),
        title=f"Week number: {week_num}",
        xlabel="Weekday",
        ylabel="Hours clocked",
        colormap=cmap,
        stacked=True,
    )

    plt.xticks(rotation=45)

    plt.legend(
        loc="upper left",
        bbox_to_anchor=(1.05, 1),
    );

### Projects per weekday

In [ ]:
columns = np.sort(data.Project.dropna().unique())
projects_per_weekday = pd.DataFrame(columns=columns, index=index)

result = product(np.sort(data.week_num.dropna().unique()), days)

for i, (week_num, weekday) in enumerate(result):
    projects_per_weekday.loc[week_num, weekday] = (
        data[(data.week_num == week_num) & (data.weekday == weekday)]
        .groupby("Project")["Duration"]
        .sum()
    )

projects_per_weekday["Total"] = projects_per_weekday.sum(axis=1)

In [ ]:
@interact
def plot_projects_per_weekday(
    week_num=projects_per_weekday.index.get_level_values("week_num").unique().tolist(),
):
    projects_per_weekday.loc[week_num, :].drop("Total", axis=1).plot.bar(
        figsize=(12, 8),
        title=f"Week number: {week_num}",
        xlabel="Weekday",
        ylabel="Hours clocked",
        colormap=cmap,
        stacked=True,
    )

    plt.xticks(rotation=45)

    plt.legend(
        loc="upper left",
        bbox_to_anchor=(1.05, 1),
    );

### Tool Usage over a Single Day

Unfortunately, the data that allows me to assess how much time I spend using particular tools expires after 1 week. Due to a misunderstanding of how the app worked and some back and forth with the dev team, I was only able to retrieve data for a single day within the observation window.

In [ ]:
# Read in timeline data
timeline_path = os.path.join(CLEAN_DATA_ABSPATH, "timeline.csv")
timeline = pd.read_csv(
    timeline_path,
    converters={
        "dt_start": pd.to_datetime,
        "dt_end": pd.to_datetime,
        "Duration": pd.to_timedelta,
    },
)

In [ ]:
# Drop unwanted column
if "idle" in timeline.columns.tolist():
    timeline.drop("idle", axis=1, inplace=True)

# Create a Date column
timeline["Date"] = timeline["dt_start"].dt.date

# Convert `Duration` to hour units
timeline["Duration"] = timeline["Duration"] / np.timedelta64(1, "h")

# Remove dates from beyond the end of the observation period
end_date = data["dt_end"].dt.date.max()
timeline.drop(
    timeline.index[timeline["Date"] - end_date > pd.to_timedelta(0)].tolist(),
    inplace=True,
)

In [ ]:
# Calculate duration of use per tool
duration_per_tool = timeline.groupby("Tool")["Duration"].sum()

# Calculate total duration across all tools
total_time = duration_per_tool.sum()

In [ ]:
# Produce Pie plot
duration_per_tool.sort_values().dropna()[duration_per_tool >= 0.05].plot.pie(
    figsize=(12, 8),
    title=f"Date: {timeline['Date'].unique().tolist()[0].strftime('%Y-%m-%d')}\nTotal = {total_time:.1f} hrs",
    ylabel="",
    colormap=cmap,
    autopct="%1.1f%%",
    pctdistance=0.85,
)

plt.legend(
    loc="upper left",
    bbox_to_anchor=(1.05, 1),
);

### Tool Usage per Project over a Single Day

In [ ]:
# Mask the time entries data
data["Date"] = data.dt_start.dt.date
masked_data = data[data.Date == timeline.Date.unique().tolist()[0]].head()

In [ ]:
# Assign a Project to each timeline entry
result = product(timeline.iterrows(), masked_data.iterrows())

for i, res in enumerate(result):
    (i, tl_row), (_, dt_row) = res

    is_within_time_frame = (
        (tl_row.dt_start - dt_row.dt_start) >= pd.to_timedelta(0)
    ) & ((tl_row.dt_end - dt_row.dt_end) <= pd.to_timedelta(0))

    if is_within_time_frame:
        timeline.loc[i, "Project"] = dt_row.Project

In [ ]:
columns = np.sort(timeline.Tool.dropna().unique())
tools_per_project = pd.DataFrame(columns=columns)

for project in np.sort(timeline.Project.dropna().unique()):
    tools_per_project.loc[project] = (
        timeline[timeline.Project == project].groupby("Tool")["Duration"].sum()
    )

tools_per_project["Total"] = tools_per_project.sum(axis=1)
tools_per_project = tools_per_project.T

In [ ]:
@interact
def plot_tools_per_project(column=tools_per_project.columns.tolist()):
    tools_per_project[column][tools_per_project[column] >= 0.05].drop(
        "Total", axis=0
    ).sort_values().dropna().plot.pie(
        figsize=(12, 8),
        title=f"Total = {tools_per_project.loc['Total', column]:.1f} hrs",
        ylabel="",
        colormap=cmap,
        autopct="%1.1f%%",
        normalize=True,
    )

    plt.legend(
        loc="upper left",
        bbox_to_anchor=(1.05, 1),
    );

### Work Schedule

In [ ]:
# Create dataframe of start and end times grouped by week number of day of the week
start_times = data.groupby(["week_num", "weekday"])["dt_start"].min()
end_times = data.groupby(["week_num", "weekday"])["dt_end"].max()
schedule_df = pd.DataFrame({"start": start_times.dt.time, "end": end_times.dt.time})

In [ ]:
pd.plotting.register_matplotlib_converters()

result = product(
    np.sort(schedule_df.index.get_level_values("week_num").dropna().unique())[::-1],
    days[::-1],
)

y_labels = []

fig, ax = plt.subplots(figsize=(12, 8))
for i, (week_num, weekday) in enumerate(result):
    try:
        ax.plot(schedule_df.loc[week_num, weekday], [i + 1, i + 1], c=cmap(i), lw=15)
        y_labels.append(f"{week_num} / {weekday}")
    except KeyError:
        pass

ax.set_yticks(range(1, 15))
ax.set_yticklabels(y_labels)
ax.set_ylabel("Week # / Day of Week")

day_range = pd.date_range("08:00:00", periods=12, freq="H").strftime("%H:%M:%S")
ax.set_xticks(day_range.values.tolist())
ax.set_xlabel("Time");